In [1]:
%env KMP_DUPLICATE_LIB_OK=TRUE 
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pyhht

from nfmd import NFMD
from utils import compute_hilbert

# Pull out common plotting settings:
from plot_params import get_plot_params
full_params, half_params = get_plot_params()
plt.rcParams.update(full_params)

env: KMP_DUPLICATE_LIB_OK=TRUE


In [2]:
# Generate an example signal
fs = 5000
t_max = 1
n = int(fs*t_max)

# Build signal
t = np.linspace(0,t_max,n)
dt = t[1]-t[0]


# Define a time-dependent phase function
f_base = 350
df_1 = 10
tau_1 = 0.5
omega_1 = lambda x: f_base + (df_1)*(1-np.exp(-x/tau_1))
amp_1 = lambda x: 1 + 0.5*np.exp(-x/3)


omega_2 = lambda x: 80-2*x
amp_2 = lambda x: 8 - 0.5*np.exp(-x)


mu = lambda x: 1.5 + 2.5*np.exp(-x/(1.5))
#mu = lambda x: 2*x-0.3*x**2
#mu = lambda t: (0.2)*np.sin(t)

# Create data with time-dependent frequency
z_1 = lambda t: amp_1(t)*np.cos(2*np.pi*omega_1(t)*t)
z_2 = lambda t: amp_2(t)*np.cos(2*np.pi*omega_2(t)*t) 

z = z_1(t) 
z += z_2(t)
z += mu(t)
#z = z/np.std(z)

In [ ]:
%%time

###############
# Nonstationary Fourier Mode Decomposition
###############

nfmd = NFMD(z/np.std(z),
            num_freqs=3,
            window_size=250,
            optimizer_opts={'lr': 4e-07},
            max_iters=100,
            target_loss=1e-3)

freqs, A, losses, indices = nfmd.decompose_signal(500)

n_freqs = nfmd.correct_frequencies(dt=dt)
n_amps = nfmd.compute_amps()
n_mean = nfmd.compute_mean()

0/4751|500/4751|1000/4751|1500/4751|

In [ ]:
%%time

###############
# Hilbert-Huang Transform
###############

emd = pyhht.emd.EMD(z, t)
imfs = emd.decompose()

imf_results = []
for imf in imfs:
    imf_results.append(compute_hilbert(imf, fs))
    
print(len(imf_results))
mean_imf = imfs[-1]

In [ ]:
fig3 = plt.figure(constrained_layout=True)
gs = fig3.add_gridspec(4, 2)

idcs = np.asarray(nfmd.mid_idcs)

##############################################
## Axes List 1 -- Top Row -- Signal Example ##
##############################################
ax1 = fig3.add_subplot(gs[0, :])
ax1.plot(t, z, lw=0.5)
ax1.set_ylabel('$z(t)$ Signal')
ax1.set_xlabel('Time (sec)')

##
## Set up lower axes
##
ax2 = fig3.add_subplot(gs[1,0])
ax3 = fig3.add_subplot(gs[1,1])
ax4 = fig3.add_subplot(gs[2,0], sharex=ax2)
ax5 = fig3.add_subplot(gs[2,1], sharex=ax3)

# Hide higher axis xlabels
plt.setp(ax2.get_xticklabels(), visible=False)
plt.setp(ax3.get_xticklabels(), visible=False)

ax2.set_title('Instantaneous Frequency')
ax3.set_title('Instantaneous Amplitude')

##########################################
## Second row -- First Mode Comparisons ##
##########################################
f_idx = 2
emd_idx = 0

ax2.plot(t, imf_results[emd_idx][1])
ax2.plot(t[nfmd.mid_idcs], n_freqs[:,f_idx])
ax2.plot(t, np.gradient(omega_1(t)*t, dt), '--k', alpha=0.75)

ax3.plot(t, imf_results[emd_idx][3])
ax3.plot(t[nfmd.mid_idcs], n_amps[:,f_idx]*np.std(z))
ax3.plot(t, amp_1(t), '--k', alpha=0.75)

# Format axes
ax2.set_ylabel(r"$\omega(t)$")
ax3.set_ylabel('$A(t)$')

ax2.set_ylim([330,380])
ax3.set_ylim([1.3,1.6])

##########################################
## Third row -- Second Mode Comparisons ##
##########################################
f_idx = 1
emd_idx = 1

ax4.plot(t, imf_results[emd_idx][1])
ax4.plot(t[nfmd.mid_idcs], n_freqs[:,f_idx])
ax4.plot(t, np.gradient(omega_2(t)*t, dt), '--k', alpha=0.75)

ax5.plot(t, imf_results[emd_idx][3])
ax5.plot(t[nfmd.mid_idcs], n_amps[:,f_idx]*np.std(z))
ax5.plot(t, amp_2(t), '--k', alpha=0.75)

# Format axes
ax4.set_ylabel(r"$\omega(t)$")
ax5.set_ylabel('$A(t)$')

ax4.set_ylim([75,82])
ax5.set_ylim([7.2,8])


##########################################
## Fourth row -- Third Mode Comparisons ##
##########################################
ax6 = fig3.add_subplot(gs[3,:])

ax6.plot(t, mean_imf, label='HHT')
ax6.plot(t[nfmd.mid_idcs], n_mean*np.std(z), label='Fourier')
ax6.plot(t, mu(t), '--k', label='Truth', alpha=0.75)

ax6.legend(bbox_to_anchor=(0.5,-1.4), loc='center', ncol=3)
ax6.set_ylabel('Mean $\mu(t)$')
ax6.set_xlabel('Time (sec)')

plt.show()

In [ ]:
# A couple troubleshooting // insightful figures:

# 1. See the difference between the correct frequencies and the predicted
fig, [ax1, ax2]=plt.subplots(2,1, sharex=True)

half_window = int(nfmd.window_size/2)
w1 = np.gradient(omega_1(t)*t, dt)[half_window:-half_window+1]
w2 = np.gradient(omega_2(t)*t, dt)[half_window:-half_window+1]

ax1.plot(t[nfmd.mid_idcs], n_freqs[:,2]-w1)
ax1.set_ylabel("$\hat{\omega}_1-\omega_1$")
ax2.plot(t[nfmd.mid_idcs], n_freqs[:,1]-w2)
ax2.set_ylabel("$\hat{\omega}_2-\omega_2$")
ax2.set_xlabel("Time")

# Take a look at the 'worst' fit result:
fig, [ax1, ax2, ax3]=plt.subplots(3,1)

ax1.plot(t[nfmd.mid_idcs], nfmd.losses)
ax1.set_ylabel("Model MSE Loss")
ax1.set_xlabel("Time")

i = np.argmax(nfmd.losses)
idcs = nfmd.indices[i]
ax2.plot(t[idcs], nfmd.window_fits[i]*np.std(z))
ax2.plot(t[idcs], z[idcs])
ax2.set_ylabel("Worst fit, \n $\hat{x}(t)-x(t)$")
ax2.set_xlabel("Time")

i = np.argmin(nfmd.losses)
idcs = nfmd.indices[i]
print(idcs, t.shape, i, len(nfmd.window_fits))
ax3.plot(t[idcs], nfmd.window_fits[i]*np.std(z))
ax3.plot(t[idcs], z[idcs])
ax3.set_ylabel("Best Fit, \n $\hat{x}(t)-x(t)$")
ax3.set_xlabel("Time")